# A Mid-Infrared Analysis of Accreeting Supermassive Black Holes 
## Utilzing Diagnostic Tools to Identify Active Galactic Nuclei (AGN)
### Part 2: Evaluating the Completness and Reliability of the AGN Selection Criteria
As any selection can suffer from incompleteness and contamination, there is a need to choose a selection that is both reliable, and complete. In this notebook, we will evaluate the completeness and reliability of the AGN selection criteria.

Given the galaxies that were selected as AGN candidates, we will compare the selection diagnostic against a truth sameple of known AGN. These known AGN have been detected by the Code Investigating GALaxy Emission (CIGALE) fitting code.

To determine the completeness and reliability of the selection criteria, we will use the following definitions:

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$

In [14]:
# Begin by importing the required packages for the project
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits

#import seaborn as sns

In [15]:
# We now need to read in the dataframes that were created in the previous notebook for all fields
# We will also need to read in the truth samples that were created from the CIGALE code

# Choose a field to work with
field = 'UDS'

# Choose the error threshold for comparison
sigma = 5

# Read in the dataframes
selections_df = pd.read_csv(field+'_combined_selection_'+str(sigma)+'_sigma.csv')



TypeError: can only concatenate str (not "int") to str

In [ ]:

# Note, FITS files by virtue of their structure are big-endian, so we need to swap the bytes to use them in 
# Pandas Dataframes, as these are little-endian by default.

# Read in the truth samples and then create dataframes from the fits files
truth_sample = fits.open(field+'_truth.fits')
truth_df=pd.DataFrame(np.array(truth_sample[1].data).byteswap().newbyteorder()) # Byteswap so that Pandas can read it
truth_df.rename(columns={'id_1':'id'}, inplace=True) # Rename the ID column so that it matches the other dataframes

In [ ]:
# Check the truth_df dataframe
truth_df.head()

,id,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_gas,bayes.stellar.m_gas_err
0,1271,9.308621e+36,1.311456e+37,1.005005e+38,5.025024e+36,22.349541,1.131768,1.787167e+10,1.366736e+09
1,1349,3.488997e+37,2.470654e+37,2.277963e+38,5.684527e+37,42.539630,16.207377,9.037132e+10,8.889398e+09
2,1363,5.513036e+37,7.235695e+37,7.594759e+38,3.797380e+37,186.449123,9.322456,4.185601e+10,2.904655e+09
3,1445,3.896829e+37,5.149345e+37,3.998763e+38,8.790852e+37,107.155606,24.810729,1.489922e+10,6.824603e+09
4,1469,3.472673e+35,8.070511e+35,1.050208e+38,5.251039e+36,22.472501,1.195830,3.117643e+10,3.605540e+09


In [ ]:
# bayes.agn.total_dust_luminosity <- AGN contribution
truth_df['bayes.agn.total_dust_luminosity']

# bayes.dust.luminosity <- stellar contribution
truth_df['bayes.dust.luminosity']

# filter the truth data based on error
truth_df = truth_df[truth_df['bayes.agn.total_dust_luminosity'] > 0]
truth_df = truth_df[truth_df['bayes.dust.luminosity'] > 0]



CIGALE uses a Bayesian approach with MCMC techniques. CIGALE allows us to analyse SED data, separatating the contributions of the AGN from the stellar-heated dust.  In the truth sample the luminosity from these two dust components can be used to derive a correct truth sample. A truth sample of AGN will be found if the luminosity contribution from the AGN is greater than 50% of the total luminosity of the system (AGN + Dust).

In [ ]:
# To find the AGN we look at the values of the AGN's luminosity, and compare it against the
# total luminosity of that galaxy. If the lumniosty of the AGN is greater than 50% of the total
# luminosity of the entire galaxy, we have an AGN.


# This will be the AGN luminosity contribution
truth_df['agn contribution'] = truth_df['bayes.agn.total_dust_luminosity']/(truth_df['bayes.agn.total_dust_luminosity'] + truth_df['bayes.dust.luminosity'])
truth_df

,id,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_gas,bayes.stellar.m_gas_err,agn contribution
0,1271,9.308621e+36,1.311456e+37,1.005005e+38,5.025024e+36,22.349541,1.131768,1.787167e+10,1.366736e+09,0.084771
1,1349,3.488997e+37,2.470654e+37,2.277963e+38,5.684527e+37,42.539630,16.207377,9.037132e+10,8.889398e+09,0.132820
2,1363,5.513036e+37,7.235695e+37,7.594759e+38,3.797380e+37,186.449123,9.322456,4.185601e+10,2.904655e+09,0.067677
3,1445,3.896829e+37,5.149345e+37,3.998763e+38,8.790852e+37,107.155606,24.810729,1.489922e+10,6.824603e+09,0.088797
4,1469,3.472673e+35,8.070511e+35,1.050208e+38,5.251039e+36,22.472501,1.195830,3.117643e+10,3.605540e+09,0.003296
...,...,...,...,...,...,...,...,...,...,...
399,21822,2.571492e+36,6.134669e+36,1.918660e+38,2.320189e+37,50.161827,6.564186,7.993137e+09,1.590875e+09,0.013225
400,21945,3.115131e+37,4.256148e+37,2.628945e+38,1.593644e+37,66.757217,7.432127,2.817712e+10,6.420238e+09,0.105940
401,21965,2.709439e+38,2.253225e+38,1.107713e+39,6.568567e+37,332.313470,28.529359,1.193109e+10,3.191506e+09,0.196527
402,22041,2.948618e+37,6.233599e+36,2.543274e+38,1.271637e+37,43.547084,2.177354,9.433791e+10,4.716896e+09,0.103893


In [ ]:

# Add a new column for known AGN
truth_df['Known AGN'] = np.where(truth_df['agn contribution'] > 0.5, 1, 0)

# This will be the AGN luminosity contribution
num_true_AGN = len(truth_df[truth_df['Known AGN'] == 1])

print("There are " + str(num_true_AGN) + " AGN that have been found by CIGALE in the " + field + " field.")

There are 38 AGN that have been found by CIGALE in the UDS field.


Now that we know the amount of AGN, we need to use the ID from the truth sample, and append this to the AGN candidates. This will allow us to compare the selection criteria against the truth sample.If the ID from the truth sample is found in the AGN candidates, then we know that the AGN candidate is also an AGN in the truth sample and thus a positive diagnostic selection

As we have two selection criteria (Lacy, Messias) we will have two seperate positive diagnostic columns for each of the criteria.



In [ ]:
# join the two dataframes
selections_df = selections_df.join(truth_df.set_index('id'), on='id')

In [ ]:
# Create a new column for positive diagnostic, this will be set to zero initally
selections_df['Positive Lacy Selection'] = 0
selections_df['Positice Messias Selection'] = 0

# We now need to test against the AGN to see if we have a positive selection


# First for Lacy, then Messias
selections_df['Positive Lacy Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Lacy Selection'] == 1), 1, 0)
selections_df['Positive Messias Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Messias Selection'] == 1), 1, 0)

In [ ]:
# Now we can do a comparison to determine the reliability and completeness of the selections
selections_df['Positive Lacy Selection'].value_counts()[1]
selections_df['Positive Messias Selection'].value_counts()[1]

8

We now can calculate the completeness of the of the selection, we do this using the criteria below

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

In [ ]:
def calculateCompleteness(df, diagnostic):
    # Calculate the completeness of the selection
    # Completeness = Positive Diagnostic / Known AGN
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Known AGN = AGN selected by CIGALE
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    known_AGN = df['Known AGN'].value_counts()[1]
    return positive_selection/known_AGN

# Calculate the completeness of the Lacy and Messias selection
lacy_completeness = calculateCompleteness(selections_df, 'Lacy')
messias_completeness = calculateCompleteness(selections_df, 'Messias')

We now can calculate the reliability of the of the selection, we do this using the criteria below

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$


In [ ]:
def calculateReliability(df, diagnostic):
    # Calculate the reliability of the selection
    # Reliability = Positive Diagnostic / Positive Selection
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Positive Selection = AGN that are selected by the selection diagnostic
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    diagnostic_selection = df[diagnostic + ' Selection'].value_counts()[1]
    return positive_selection/diagnostic_selection

# Calculate the reliability of the Lacy and Messias selection
lacy_reliability = calculateReliability(selections_df, 'Lacy')
messias_reliability = calculateReliability(selections_df, 'Messias')


selections_df[selections_df['Known AGN'] == 1]


,id,x,y,ra,dec,SEflags,iso_area,fap_Ksall,eap_Ksall,apcorr,...,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_gas,bayes.stellar.m_gas_err,agn contribution,Known AGN,Positive Lacy Selection,Positice Messias Selection,Positive Messias Selection
8,1702,2647.499,561.096,34.318493,-5.293310,3,445.0,10.948310,0.073619,1.035271,...,5.221579e+35,0.970987,0.103054,2.246241e+09,3.496130e+08,0.570621,1,1,0,0
39,3378,4017.520,1077.630,34.261166,-5.271775,2,440.0,27.337360,0.068864,1.057582,...,3.005983e+35,0.455382,0.022769,3.272594e+10,1.636297e+09,0.793511,1,1,0,0
41,3417,3899.422,1069.488,34.266109,-5.272116,3,209.0,7.678557,0.070485,1.065928,...,1.619492e+37,67.620403,4.846995,1.213226e+10,2.941439e+09,0.591324,1,1,0,0
43,3526,1595.463,1132.784,34.362522,-5.269492,2,548.0,8.635911,0.068476,1.019660,...,2.697256e+36,4.073085,0.638178,8.279249e+09,7.366358e+08,0.729408,1,1,0,0
62,4484,3653.196,1382.952,34.276413,-5.259056,3,458.0,43.530710,0.057099,1.068757,...,5.285356e+38,307.414267,169.952357,1.500498e+10,7.091708e+09,0.516825,1,1,0,1
67,4780,3582.055,1464.147,34.279392,-5.255673,2,953.0,34.560030,0.056658,1.021699,...,1.140173e+36,1.844207,0.187572,1.266872e+10,1.075049e+09,0.508959,1,1,0,0
79,5680,1409.891,1674.965,34.370289,-5.246900,3,653.0,56.415940,0.051231,1.046818,...,2.100952e+37,8.012539,4.416970,9.420468e+10,4.710234e+09,0.718438,1,1,0,1
88,6097,2260.268,1756.514,34.334705,-5.243499,3,562.0,28.142880,0.051722,1.045044,...,4.978732e+37,10.811043,11.946415,4.867405e+10,1.300372e+10,0.694080,1,1,0,1
89,6226,4643.729,1760.463,34.234970,-5.243315,3,148.0,2.716645,0.051071,1.076544,...,2.804430e+37,58.216060,6.903392,3.209016e+09,2.725897e+09,0.655437,1,1,0,0
94,6442,3183.748,1802.411,34.296062,-5.241581,0,191.0,2.545368,0.051393,1.063765,...,5.791686e+37,39.771962,13.472783,6.533226e+09,3.424517e+09,0.677436,1,1,0,0


## Conclusion
Bringing this all together we can see both the completeness and relaibility of each of the diagnostics. Below we see the outputs of the completeness and reliability tests for our Lacy and Messias diagnostics. 

In [ ]:
print("The completeness of the Lacy selection is " + str(round(lacy_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Lacy selection is " + str(round(lacy_reliability*100, 2))+"% in the " + field + " field.")
print("\n")
print("The completeness of the Messias selection is " + str(round(messias_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Messias selection is " + str(round(messias_reliability*100, 2))+"% in the " + field + " field.")

The completeness of the Lacy selection is 97.06% in the UDS field.
The reliability of the Lacy selection is 8.8% in the UDS field.


The completeness of the Messias selection is 23.53% in the UDS field.
The reliability of the Messias selection is 14.81% in the UDS field.
